<img src="./imgs/banner.png" align="center"/>

## ¡Te damos la bienvenida a la experiencia ATGenomics! 

### En el taller "Exploración de Datos Biológicos con Pandas" aprenderemos a usar diversas funciones de la biblioteca pandas. El objetivo principal es que aprendas a realizar un análisis exploratorio de tus datos bioinformáticos y te diviertas en el proceso. 

# 2. Cuadernillo de ejercicios

<img src="./imgs/line.png" align="left"/>

# ¿Qué preguntas biologicas podemos abordar con pandas?
En la enciclopedia de genes y genomas de Kyoto ([KEGG](https://www.genome.jp/kegg/) por sus siglas en inglés), están depositadas cientos de rutas metabólicas y mapas celulares que se han construido mediante la integración de técnicas moleculares, genéticas, bioquímicas y de otras disciplinas.

## Unas rutas como ejemplo

A continuación enlistamos un par de rutas metabólicas que podrían ser de interés y que podemos analizar usando **Pandas**
* [hsa04930](https://www.genome.jp/pathway/hsa04930) - Type II diabetes mellitus
* [hsa05010](https://www.genome.jp/pathway/hsa05010) - Alzheimer disease
* [hsa05012](https://www.genome.jp/pathway/hsa05012) - Parkinson disease
* [hsa05030](https://www.genome.jp/pathway/hsa05030) - Cocaine addiction
* [hsa05144](https://www.genome.jp/pathway/hsa05144) - Malaria
* [hsa05210](https://www.genome.jp/pathway/hsa05210) - Colorectal cancer
* [hsa05224](https://www.genome.jp/pathway/hsa05224) - Breast cancer

Cada ruta consta de genes, proteínas, compuestos, reacciones, e interacciones entre cada elemento.

## Exploremos estas rutas con Pandas!
<img src="./imgs/img_pandas2.jpeg" align="center"/>

## 1. Cargamos los modulos necesarios

In [1]:
import os                                 # Con este módulo podemos manipular elementos del sistema operativo
import gzip                               # Este módulo lo emplearemos para manejar archivos comprimidos
import wget                               # wget & requests nos sirven para descargar archivos e interactuar con sitios web
import requests                           # 
import pandas as pd                       # nuestro módulo principal
import seaborn as sns                     # usaremos seaborn & matplotlib para graficar los resultados que obtengamos con pandas
import matplotlib.pyplot as plt           # 
from PIL import Image                     # finalmente, usaremos wordcloud y un par de módulos adicionales para analizar nuestras rutas
from wordcloud import WordCloud           #
from wordcloud import STOPWORDS           #
from wordcloud import ImageColorGenerator #

## 2. Descargamos el archivo gff que hemos empleado a lo largo del taller

<div class="alert alert-warning">Este paso es opcional ya que hemos trabajado con el mismo archivo desde el inicio, pero lo incluimos por si quieres realizar los ejercicios iniciando en este punto</div>

In [2]:
os.makedirs('data', exist_ok=True)
url      = 'http://ftp.ensembl.org/pub/release-104/gff3/homo_sapiens/Homo_sapiens.GRCh38.104.gff3.gz'
request  = requests.get(url, allow_redirects=True)
filename = "./data/Homo_sapiens.GRCh38.104.gff3.gz"
open(filename, "wb").write(request.content)

43154053

## 3. Construimos un dataframe con pandas indicando el nombre de las columnas con base en la especificación del formato gff

* Column 1: "seqid"
* Column 2: "source"
* Column 3: "type"
* Columns 4 & 5: "start" and "end"
* Column 6: "score"
* Column 7: "strand"
* Column 8: "phase"
* Column 9: "attributes"

In [3]:
df = pd.read_csv("data/Homo_sapiens.GRCh38.104.gff3.gz", sep="\t", comment="#", dtype="str", header=None, names=["seqid","source","type","start","end","score","strand","phase","attributes"])

## 4. Eliminamos las entradas en el archivo gff que correspondan a secuencias que no tienen aún una localizacion genómica

1. Primero haremos una copia del dataframe original, así, si cometemos un error podemos regresar al punto inicial *#bestpractices*
2. Posteriormente eliminamos las entradas que contengan `KI` o `GL` en la columna `seqid`
3. Reiniciamos el índice de nuestro nuevo dataframe

In [4]:
filtered_df = df.copy()
filtered_df = filtered_df[~filtered_df['seqid'].str.contains('[KI, GL]')].reset_index(drop=True)

## 5. Conservamos únicamente las entradas que correspondan al tipo `gene`

Recuerda que es recomendable construir una copia del dataframe original

In [5]:
genes = filtered_df.copy()
genes = genes[genes['type'] == 'gene'].reset_index(drop=True)

## 6. Construimos un nuevo dataframe pero únicamente con la novena columna del dataset (`attributes`)

1. Nuestro nuevo dataframe requiere los nombres de las columnas, las cuales están almacenadas en el diccionario `attribute_names`
2. Para no alterar el dataframe `genes` construimos una copia de dicho dataframe
3. Transformamos la columna 9 en un dataframe usando `";"` como separador

In [6]:
attribute_names = {0:"id",1:"name",2:"biotype",3:"description",4:"gene_id",5:"logic_name",6:"version"}
attributes = genes.copy()
attributes = attributes["attributes"].str.split(";",expand=True).rename(attribute_names,axis=1)

## 7. Construimos *otro* dataframe fusionando los dataframes `genes` & `attributes`

1. Dado que en el dataframe `attributes` ya tenemos la información de la columna 9, no necesitamos la columna `attributes` en el dataframe `genes`

In [7]:
genes_attributes = pd.concat([genes.drop("attributes",axis=1),attributes], axis=1)

## 8. Limpiamos el dataframe para quitarnos de strings no deseados

1. Usando una expresión regular, eliminamos todo aquello que esté antes de un signo `"="` en cada columna
2. Usando otra expresión regular, eliminamos el string `gene:` en cada columna
3. Usando una expresión regular más compleja, eliminamos todo aquello que inicie con `"[Source"` en cada columna

In [8]:
genes_attributes = genes_attributes.replace('.*=', '', regex=True).replace('gene:', '', regex=True).replace("\[Source.*","",regex=True)

## 9. Nuestro dataframe no está ordenado, de modo que debemos ordenarlo adecuadamente

### ¿Cómo ordenamos los cromosomas?

* No podemos ordenar nuestros cromosomas de forma numérica porque tenemos 3 secuencias que no son números (`X`,`Y` & `MT` )
* No podemos ordenar nuestros cromosomas de forma numérica porque en el dataframe la columna `seqid` es de tipo `str`

#### Lo que si podemos hacer es ordenar nuestro dataframe usando una lista!

1. Construimos una nueva copia temporal del dataframe porque #sanitycheck
2. Construimos una copia adicional del dataframe, la cúal iremos alimentando con las entradas de cada cromosoma. Este dataframe contendrá inicialmente las entradas correspondientes únicamente al cromosoma 1
3. Construimos una lista con los cromosomas ordenados
4. Iteramos sobre nuestra lista, tomando los elementos del dataframe temporal que correspondan a cada cromosoma y los pasamos a nuestro dataframe final (paso 9.2)
5. Verificamos los resultados

In [9]:
genes_tmp = genes_attributes.copy()
genes_attributes_sorted = genes_tmp[genes_tmp["seqid"]=="1"]
chromosome_list = ["2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","X","Y","MT"]
for chromosome in chromosome_list:
    genes_chrom = genes_tmp[genes_tmp["seqid"]==chromosome]
    genes_attributes_sorted = pd.concat([genes_attributes_sorted,genes_chrom],axis=0)
display(pd.unique(genes_attributes["seqid"]))
display(pd.unique(genes_attributes_sorted["seqid"]))

array(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19',
       '2', '20', '21', '22', '3', '4', '5', '6', '7', '8', '9', 'MT',
       'X', 'Y'], dtype=object)

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', 'X',
       'Y', 'MT'], dtype=object)

## 10. Cambiamos el tipo de datos en las columnas 4 y 5

1. En el paso 3, le indicamos a pandas que todas las columnas deben ser de tipo `str`, sin embargo, las columnas 4,5 & 6 deben ser de tipo numérico para que podamos sacarles el mayor provecho

In [10]:
genes_attributes_sorted["start"] = pd.to_numeric(genes_attributes_sorted["start"])
genes_attributes_sorted["end"]   = pd.to_numeric(genes_attributes_sorted["end"])

# Contacto

<img src="./imgs/line.png" align="left"/>

### Este curso es llevado a ti por [ATGenomics](https://atgenomics.com/), una iniciativa mexicana dedicada a la divulgación, promoción y capacitación en bioinformática.

### [@solnavss](https://twitter.com/solnavss) / [@dianolasa](https://twitter.com/dianolasa) / [@zorbax](https://twitter.com/zorbax) / [@vflorelo](https://twitter.com/vflorelo)